In [ ]:
import numpy as np
import cv2
import torch
import pickle


In [ ]:
# upload dataset and unzip it (make sure data.yml is already modified as per our need)
!unzip 'Cars - Overhead View.v3-roboflow-accurate-model.yolov5pytorch.zip'

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17067, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 17067 (delta 24), reused 27 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17067/17067), 15.69 MiB | 16.46 MiB/s, done.
Resolving deltas: 100% (11718/11718), done.


In [ ]:
# training the model
!python "/content/yolov5/train.py" --img 640 --batch 16 --epochs 20 --data "/content/data.yaml" --cfg "/content/yolov5/models/yolov5s.yaml" --weights yolov5s.pt --name yolov5-custom --device 0

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-12-03 14:27:08.685275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-03 14:27:08.707612: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-03 14:27:08.714450: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visua

In [ ]:
global width, height
width, height = 107, 48

In [ ]:
def get_model(weights_path):
  model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)
  model.conf = 0.6  # Set confidence threshold
  return model

In [ ]:
def fill_global_positons_list_n_return_img(image, frame, pos_pickle_path):
  with open(pos_pickle_path, 'rb') as f:
    posList = pickle.load(f)
  global width, height
  position_list = []
  # poslist_img = np.zeros((720, 1100, 1), dtype=np.uint8)
  intensity = 255
  target_size = (1100, 720)
  image = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
  frame = cv2.resize(frame, target_size, interpolation=cv2.INTER_LINEAR)
  #cv2.imwrite("1d_image_resized.jpg", image)
  vacant_count=0
  occupied_count=0
  for idx, pos in enumerate(posList):
    x,y = pos
    #y += 50
    # poslist_img[y:y+height,x:x+width] = intensity
    imgCrop = image[y:y+height,x:x+width]
    mask = (imgCrop == 255).astype(np.uint8)
    count = cv2.countNonZero(mask)
    if count > 1500:
      position_list.append(1)
      occupied_count+=1
      color = (0, 255, 255)
      cv2.rectangle(frame, (x, y), (x+width, y+height), color, 2)
    else:
      position_list.append(0)
      vacant_count+=1
      color = (0, 100, 0)
      cv2.rectangle(frame, (x, y), (x+width, y+height), color, 2)
  cv2.putText(frame,f"Vacant: {vacant_count} | Occupied: {occupied_count}" ,(10, 30),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 255, 255),2)

    # print(idx)
  # cv2.imwrite("frame_output.jpg", frame)
  return frame


In [ ]:
def process_video(vid_path, output_path, weights_path, pos_pickle_path):
  model = get_model(weights_path)

  # Open the video file
  cap = cv2.VideoCapture(vid_path)
  if not cap.isOpened():
      print("Error: Unable to open video.")
      return

  # Get video properties (fps, frame width, and height)
  fps = int(cap.get(cv2.CAP_PROP_FPS))
  frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  print(frame_width, frame_height)

  # Define VideoWriter to save the processed video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4 video
  out = cv2.VideoWriter(output_path, fourcc, fps, (1100, 720)) #(1100, 820) replaced (frame_width, frame_height)

  # Process the video frame by frame
  y=0
  while y<240:
      y+=1
      one_d_boxes = np.zeros((frame_height, frame_width, 1), dtype=np.uint8)
      ret, frame = cap.read()
      if not ret:
          break  # Stop if the video ends

      # Run inference (pass the frame through the YOLO model)
      results = model(frame)

      # Extract predictions (bounding boxes, confidence, and class labels)
      predictions = results.xyxy[0].cpu().numpy()  # [x1, y1, x2, y2, confidence, class]

      # Draw bounding boxes on the frame
      for *box, conf, cls in predictions:
          x1, y1, x2, y2 = map(int, box)
          color = (0, 255, 0)  # Color for the bounding box (green for 'car')
          intensity = 255

          # Draw rectangle (bounding box)
          # cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
          one_d_boxes[y1:y2, x1:x2] = intensity
      cv2.imwrite("1d_image.jpg", one_d_boxes)
      frame = fill_global_positons_list_n_return_img(one_d_boxes, frame, pos_pickle_path)
      # break
      # Write the processed frame to the output video
      out.write(frame)

In [ ]:
# main code to execute
vid_path = "/content/ParkingLot.mp4"
weights_path = "/content/yolov5/runs/train/yolov5-custom/weights/best.pt"
#weights_path= "/content/best weights yolov5.pt"
output_path = "/content/output.mp4"
pos_pickle_path = "/content/CarParkPos"
process_video(vid_path, output_path, weights_path, pos_pickle_path)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-3 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/commo

1100 720


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.am